In [1]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import accuracy_score

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts

#TS Fresh Parameter Settings
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.feature_extraction import MinimalFCParameters
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh.feature_extraction import extract_features

# Set display options to show all rows and columns
pd.set_option('display.max_rows', 50)  # Show rows
pd.set_option('display.max_columns', 160)  # Show columns

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))


In [2]:
load_path = '/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/Baseline_Extensions/Gen_New_Features/generated_feats/'
data_types = ['Wake','N1', 'N2', 'N3', 'REM']
loaded_data = {}


for data_type in data_types:
    print(data_type)
    X_y_groups = {}
    
    X = pd.read_hdf(load_path + data_type +  'six_second_max_freq_stats_df.h5', key='df', mode='r')
    y = pd.read_hdf(load_path + data_type +  '_y.h5', key='df', mode='r') 
    groups = pd.read_hdf(load_path + data_type +  '_groups.h5', key='df', mode='r')
    X, y , groups = X.reset_index(drop = True) , y.reset_index(drop = True) , groups.reset_index(drop = True)

    #Transform the X into TS_Fresh Features___
    # 1. Convert to TS_Fresh format Dataframe 
    ts_fresh_df = format_eeg_data.convert_sktime_df_to_ts_fresh_format(X, ts_cols = list(X.columns))
    
    # 2. Extract TS_Fresh Features from the dataframe
    settings = MinimalFCParameters()
    extracted_ts_fresh_df = extract_features(ts_fresh_df, column_id = 'id' , column_sort = 'time',  default_fc_parameters=settings)

    # 3. Asign extract_ts_fresh_df to the variable X
    X = extracted_ts_fresh_df.copy()
    print(len(X.columns))
    #Drop columns where all values are NA 
    X = X.dropna(axis = 1)
    print(len(X.columns))

    #Drop columns where all values are the same
    # Find columns where all values are the same
    same_value_columns = X.columns[X.nunique() == 1]
    # Drop columns with the same values
    X = X.drop(columns=same_value_columns)
    print(len(X.columns))

    #Generate the three dictionaries per data type _________________________________________________________________________________
    #Generate regional features dict per data type 
    #Generate a region to features dictionary - this will enable us to run expts regionally as before
    regional_features_dict = {}
    region_channel_dict = constants.region_to_channel_dict
    regions = list(region_channel_dict.keys())
    for region in regions:
        region_features = [col for col in X.columns if region + '_' in col]
        if len(region_features) > 0 : 
            regional_features_dict[region] = region_features

    # 2. #Create the combined regions dictionary
    regions = list(regional_features_dict.keys())
    combined_regions_features_dict = {}
        
    for i, region_1 in enumerate(regions):
        for region_2 in regions[i+1:]:
            new_key = region_1 + '_' + region_2
            combined_regions_features_dict[new_key] = regional_features_dict[region_1] + regional_features_dict[region_2]

    #3. Use all of the features
    all_data_dict = {'All_regions' : list(X.columns) , 'All_regions_2' : list(X.columns)}
       
    X_y_groups['X'] = X
    X_y_groups['y'] = y
    X_y_groups['groups'] = groups
    
    X_y_groups['regions_dict'] = regional_features_dict
    X_y_groups['combined_regions_dict'] = combined_regions_features_dict
    X_y_groups['all_feats_dict'] = all_data_dict

    loaded_data[data_type] = X_y_groups

Wake


14-Sep-23 12:24:00 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 119/119 [00:01<00:00, 71.47it/s]


1040
1040
1040
N1


14-Sep-23 12:24:03 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 120/120 [00:02<00:00, 56.40it/s]


1040
1040
1038
N2


...

1040
1040
1021
N3


14-Sep-23 12:24:11 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 119/119 [00:02<00:00, 49.69it/s]


1040
1040
1017
REM


14-Sep-23 12:24:15 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 120/120 [00:02<00:00, 53.28it/s]


1040
1040
1030


### Run all expts

In [3]:
t1 = time.time()

all_expt_results = {}

for expt_num in [1,2,3,4] :
    t3 = time.time()
    expt_results = {}
    for data_type in data_types:
        X_y_groups = loaded_data[data_type]
        
        X = X_y_groups['X']
        y = X_y_groups['y'] 
        groups = X_y_groups['groups'] 
            
        #1.Generate expt specific X,y,groups
        X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, expt_num )
    
        results_df_regional = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt,  {'RF' : RandomForestClassifier(random_state = 5) , 'DT' : DecisionTreeClassifier() , 'Ada_B' : AdaBoostClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = X_y_groups['regions_dict'], random_states = [1,2] )
        print('regional done...')
        results_df_regions_combined = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt,  {'RF' : RandomForestClassifier(random_state = 5) , 'DT' : DecisionTreeClassifier() , 'Ada_B' : AdaBoostClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = X_y_groups['combined_regions_dict'] , random_states = [1,2] )
        print('regions combined done...')
        results_df_all_feats = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt,  {'RF' : RandomForestClassifier(random_state = 5) , 'DT' : DecisionTreeClassifier() , 'Ada_B' : AdaBoostClassifier(random_state = 5)} , return_df = True , subset_names_and_cols = X_y_groups['all_feats_dict'], random_states = [1,2] )
        
        expt_results[data_type] = {'regional' : results_df_regional , 'regions_combined' : results_df_regions_combined , 'all_feats' : results_df_all_feats}
    
    all_expt_results[expt_num] = expt_results
    #Saving Results 
    joblib.dump(all_expt_results, 'TS_Fresh_minimal_frequency_feats_results.pkl')
    t4 = time.time()
    loop_time = t4-t3
    print(str(loop_time))

t2 = time.time()

t2-t1

regional done...
regions combined done...
regional done...
regions combined done...
regional done...
regions combined done...
regional done...
regions combined done...
regional done...
regions combined done...
883.6553180217743
regional done...
regions combined done...
regional done...
regions combined done...
regional done...
regions combined done...
regional done...
regions combined done...
regional done...
regions combined done...
1069.1420826911926
regional done...
regions combined done...
regional done...
regions combined done...
regional done...
regions combined done...
regional done...
regions combined done...
regional done...
regions combined done...
1083.3208634853363
regional done...
regions combined done...
regional done...
regions combined done...
regional done...
regions combined done...
regional done...
regions combined done...
regional done...
regions combined done...
1148.59006357193


4184.70952963829